# Greenhouse gas emissions of pedestrian car traffic in Helsinki

In [ ]:
import math
import re
import pandas as pd
import numpy as np

import plotly
import plotly.graph_objs as go
import cufflinks as cf

plotly.offline.init_notebook_mode(connected=True)
cf.set_config_file(offline=True)

%reload_ext autoreload
%autoreload 2
import lipasto

In [ ]:
muni = lipasto.get_liisa_muni_data()
out = muni.copy().xs('Helsinki')
out['mileage'] /= 1000000
display(out)

In [ ]:
out.reset_index(['Vehicle', 'Road']).iplot(
    title='GHG emissions of road traffic in Helsinki in 2017',
    kind='pie', labels='Vehicle', values='CO2e'
)

In [ ]:
mileage_share_per_engine_type = lipasto.get_mileage_per_engine_type()
display(mileage_share_per_engine_type.style.format("{:.2%}"))

In [ ]:
def generate_electricity_unit_emission_series():
    years = pd.period_range(start=2018, end=2035, freq='Y')
    co2e_by_year = pd.Series(index=years, dtype=float)
    co2e_by_year.loc['2018'] = 100  # g CO2e/kWh
    co2e_by_year.loc['2030'] = 70   # g CO2e/kWh
    co2e_by_year.loc['2035'] = 45   # g CO2e/kWh
    co2e_by_year.interpolate(inplace=True)
    return co2e_by_year

def estimate_bev_unit_emissions(unit_emissions, year):
    ENERGY_CONSUMPTION = (
        ('Highways', 0.2),   # kWh/km
        ('Urban', 0.17)
    )
    rows = []
    electricity_ghg = generate_electricity_unit_emission_series()
    for road_type, kwh in ENERGY_CONSUMPTION:
        rows.append({
            "Engine": 'electric',
            "Road": road_type,
            "Car year": 2018,
            "CO2e": kwh * electricity_ghg[str(year)],
            "Class": "EURO 6"
        })
    return unit_emissions.append(pd.DataFrame(rows), ignore_index=True, sort=True)

car_unit_emissions = lipasto.get_car_unit_emissions()
car_unit_emissions.xs('gasoline').xs('Urban')[['Car year', 'CO2e']].iplot(
    kind='line', x='Car year',
    layout=dict(
        yaxis=dict(rangemode='tozero', title='g CO2e/km', tickformat=',.0f', fixedrange=True),
        xaxis=dict(title='Car model year', fixedrange=True),
        title='Gasoline engine GHG unit emissions in urban driving'
    )
)


In [ ]:
def calculate_co2e_per_engine_type(ratios, unit_emissions):
    # yearly passenger car kms in Helsinki
    km_in_helsinki = muni.xs('Helsinki').xs('Cars')['mileage']
    # df = ratios of passenger cars on the roads by engine type
    df = ratios.xs('Cars')
    roads = ('Urban', 'Highways')
    df = pd.concat([df * km_in_helsinki[road] for road in roads], keys=roads, names=['Road'])
    out = df * unit_emissions

    out /= 1000000000  # convert to kt (CO2e)
    return out


def bass_diffuse(t, p, q):
    e1 = math.e ** (-(p + q) * t)
    res = ((p + q) ** 2) / p
    res *= e1 / ((1 + q / p * e1) ** 2)
    return res


def estimate_mileage_ratios(year):
    # Assume BEV share is increasing according to the Bass diffusion model
    # and that increase in share comes equally out of petrol and diesel engines
    # starting from the most polluting engine classes.
    df = mileage_share_per_engine_type.drop(columns='Sum')
    bev = df.loc['Cars', 'electric']
    bev_share_start = bev_share = bev['EURO 6']
    for t in range(year - 2018):
        bev_share *= 1 + bass_diffuse(t, 0.05, 0.38) * 8.5
    bev['EURO 6'] = bev_share

    share_change = bev_share - bev_share_start
    sums = df.loc['Cars'].sum(axis='columns')
    diesel_share = sums['diesel'] / (sums['diesel'] + sums['gasoline'])
    share_left = dict(diesel=share_change * diesel_share)
    share_left['gasoline'] = share_change - share_left['diesel']
    
    for i in range(0, 6 + 1):
        key = 'EURO %d' % i
        for eng in ('diesel', 'gasoline'):
            if not share_left[eng]:
                continue

            val = df.loc['Cars', eng][key]
            decrease = min(val, share_left[eng])
            df.loc['Cars', eng][key] -= decrease
            share_left[eng] -= decrease

    return df

def estimate_unit_emissions(year):
    df = car_unit_emissions.reset_index()
    df = estimate_bev_unit_emissions(df, year)
    df.set_index(['Engine', 'Road'], inplace=True)
    df = df.groupby(['Road', 'Engine', 'Class']).mean()['CO2e'].unstack('Class')
    return df


yearly_dfs = []
for year in range(2018, 2036):
    share = estimate_mileage_ratios(year)
    unit_emissions = estimate_unit_emissions(year)
    co2e = calculate_co2e_per_engine_type(share, unit_emissions)

    share = share.sum(axis='columns').xs('Cars')
    co2e = co2e.sum(level='Engine').sum(axis='columns')
    out = pd.concat([share, co2e], axis=1, keys=['Share', 'CO2e'], sort=True)
    out.index.name = 'Engine'
    yearly_dfs.append((year, out))

out = pd.concat([x[1] for x in yearly_dfs], axis=0, keys=[x[0] for x in yearly_dfs], names=['Year'])
# Show only some engine types
out = out.reindex(('gasoline', 'diesel', 'electric'), level='Engine')
display(out.style.format("{:.2%}").format({'CO2e': '{:.0f}'}))
out = out.unstack()

In [ ]:
fig1 = out.drop('CO2e', axis=1)['Share'].iplot(kind='line', asFigure=True)
fig2 = go.Bar(x=out.index, y=out['CO2e'].sum(axis='columns'), yaxis='y2', name='Car traffic CO₂e.', opacity=0.2)

shapes = [{
    'type': 'line',
    'xref': 'x',
    'yref': 'y2',
    'x0': out.index[0] - 0.5,
    'y0': 79,
    'x1': out.index[-1] + 0.5,
    'y1': 79,
    'line': {
        'color': 'green',
        'width': 3,
    },
}]

layout = go.Layout(
    xaxis=dict(title='year'),
    yaxis1=dict(overlaying='y2', tickformat=',.0%', rangemode='nonnegative'),
    yaxis2=dict(side='right', tickformat=',.0f', hoverformat='.0f', rangemode='nonnegative', title='kt'),
    barmode='overlay',
    shapes=shapes,
    title="The effect on cars' GHG emissions in Helsinki when the share of electric cars increase"
)


fig = go.Figure(data=(fig2,) + fig1.data, layout=layout)

plotly.offline.init_notebook_mode(connected=True)
plotly.offline.iplot(fig)
